In [ ]:
Pour se connecter à ce notebook 
Besoin de lancer  l'image singularity de : _Containers/shortcakelight.sif

https://github.com/rnakato/ShortCake/tree/master
Est conçu spécifiquement pour les analyses de données single cell

Pour lancer l'image singularity et pouvoir se connecter aux notebooks jupyter:
srun --pty bash
singularity exec 6_Containers/shortcakelight.sif jupyternotebook.sh

puis faire le lien ssh sur l'ordi avec cette commande  que l'on ecrit dans le terminal local: 
ssh -A -t -t vgoupille@genossh.genouest.org -L 8888:localhost:8888 ssh cl1n030 -L 8888:localhost:8888
modifier le port (numero et noeud si besoin)

si probleme de connection 

ls -a
rm -r .vscode-server
puis se remet sur vscode 
et refait la liaison avec ssh remote (et reinstaller les extensions...)

In [ ]:
Library strategy: microSPLiT
The data preprocessing and alignment was performed using a modified SPLiT-seq pipeline (https://github.com/Alex-Rosenberg/split-seq-pipeline). Modifications: using STAR with the splicing isoform detection swiwwe usededt the hioutghoutest-scored multimapping re. We also keptwening a ctional count based on the number igninggood alignments, since bacterial genomes are known to contain overlapping CDSs.
Genome_build: ASM904v1.45 and ASM80076v1.37 from EnsemblBacteria
Supplementary_files_format_and_content: Cell by gene matrix (with cells above threshold 200 UMI/cell); Gene names; Cell annotations (barcode and well for heat shock data, OD for B.subtilis growth curve). For B. Subtilis growth curve, only mRNA are included in the cell-by-gene matrix.
Supplementary_files_format_and_content: For the heat-shock data, barcodes in wells 1-24 belong to heat-shocked cells and in wells 25-48 belong to control cells.

In [ ]:
Ce texte décrit les stratégies et les méthodes utilisées pour traiter et aligner les données issues de l’expérience scRNA-seq réalisée avec la méthode microSPLiT.
	1.	Stratégie de séquençage (Library strategy) :
	•	La méthode utilisée est microSPLiT, qui est une adaptation de la technique SPLiT-seq, permettant de réaliser du scRNA-seq sans nécessiter d’isolation physique des cellules.
	2.	Prétraitement et alignement des données :
	•	L’alignement des séquences a été effectué avec une version modifiée du pipeline SPLiT-seq disponible sur GitHub (https://github.com/Alex-Rosenberg/split-seq-pipeline).
	•	Modifications apportées :
	•	Utilisation de STAR (un logiciel d’alignement des lectures RNA-seq) avec une détection des isoformes d’épissage.
	•	Sélection des meilleurs alignements multimapping selon un critère de score élevé.
	•	Conservation d’un compte fonctionnel basé sur le nombre d’alignements de bonne qualité, ce qui est pertinent pour les génomes bactériens où plusieurs CDSs (séquences codantes) peuvent se chevaucher.
	3.	Références génomiques utilisées :
	•	Les génomes de référence utilisés pour l’alignement sont :
	•	ASM904v1.45
	•	ASM80076v1.37
	•	Ces versions proviennent de la base de données EnsemblBacteria.
	4.	Fichiers supplémentaires et leur contenu :
	•	Les fichiers supplémentaires fournis incluent :
	•	Matrice cellule-gène (contenant uniquement les cellules ayant un minimum de 200 UMI/cellule).
	•	Noms des gènes.
	•	Annotations des cellules :
	•	Pour les données du stress thermique (heat-shock) : les cellules sont annotées par code-barres et puits.
	•	Pour la courbe de croissance de Bacillus subtilis : les cellules sont annotées selon leur densité optique (OD). Seuls les ARNm (mRNA) sont inclus dans la matrice cellule-gène.
	5.	Organisation des cellules dans l’expérience de stress thermique :
	•	Les cellules sont disposées dans 48 puits, avec une distinction entre conditions :
	•	Puits 1-24 : cellules soumises au stress thermique.
	•	Puits 25-48 : cellules témoins (non soumises au stress thermique).

replica 1 : M11 => B. subtilis PY79 (OD0.5-OD3.2)

replica 2 : M14 => B. subtilis PY79 (OD0.5-OD6.0)

other plate : M15 =>	B. subtilis PY79 + E. coli MW1255

#Ici utilisé les données de sorties de StarSOLO qui sont sur le site : https://osf.io/kjfbz/files/osfstorage?view_only=#
GEO60  et GEO61



# info sur les objets de type Anndata
https://anndata.readthedocs.io/en/latest/tutorials/notebooks/getting-started.html

# info sur l'utilisation de Scanpy 
https://scanpy.readthedocs.io/en/stable/tutorials/basics/clustering.html

In [13]:
# Load libraries and abreviate them
import pandas as pd
import scanpy as sc
import numpy as np
from scipy.sparse import csr_matrix
import pprint as pp


# Définition du chemin des fichiers
data_dir = "7_Article/data/data_osf/GEO660/"

# Chargement des barcodes et des gènes
barcodes = pd.read_csv(f"{data_dir}barcodes.tsv", header=None, sep="\t")[0]
features = pd.read_csv(f"{data_dir}features.tsv", header=None, sep="\t")
matrix = pd.read_csv(f"{data_dir}matrix.mtx", skiprows=0, header=None, sep=" ")
#Preview the data
print ( "barcode file :")
print(barcodes.head())
print ( "genes names file :")
print(features.head())
print ( "matrix file :")
print(matrix.head())

# Chargement de la matrice en format sparse
#adata = sc.read_mtx(f"{data_dir}matrix.mtx").T  # Transposition pour correspondre aux dimensions attendues
adata = sc.read_mtx(f"{data_dir}UniqueAndMult-Uniform.mtx").T  # Transposition pour correspondre aux dimensions attendues





# Attribution des noms aux cellules (obs) et aux gènes (var)
adata.obs_names = barcodes
adata.var_names = features[1]  # Deuxième colonne pour les noms de gènes

# Ajout des métadonnées minimales (n'est pas obligatoire)
adata.obs["barcode"] = adata.obs.index
adata.var["gene_id"] = adata.var.index

# Conversion explicite en format sparse pour optimiser la mémoire
adata.X = csr_matrix(adata.X)

# Affichage des informations principales
print(adata)  
#print(adata.obs.head())  # Vérification des cellules
#print(adata.var.head())  # Vérification des gènes

pp.pprint(adata.obs.head())  # Vérification des cellules
pp.pprint(adata.var.head())  # Vérification des gènes


/tmp/ipykernel_15426/4188076402.py:15: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  matrix = pd.read_csv(f"{data_dir}matrix.mtx", skiprows=0, header=None, sep=" ")


barcode file :
0    AAACATCG_AAACATCG_AAACGATA
1    AACAACCA_AAACATCG_AAACGATA
2    AACCGAGA_AAACATCG_AAACGATA
3    AACGCTTA_AAACATCG_AAACGATA
4    AACGTGAT_AAACATCG_AAACGATA
Name: 0, dtype: object
genes names file :
           0          1                2
0  BSU_00010  BSU_00010  Gene Expression
1  BSU_00020  BSU_00020  Gene Expression
2  BSU_00030  BSU_00030  Gene Expression
3  BSU_00040  BSU_00040  Gene Expression
4  BSU_00050  BSU_00050  Gene Expression
matrix file :
                0       1           2        3        4
0  %%MatrixMarket  matrix  coordinate  integer  general
1               %     NaN         NaN      NaN      NaN
2            4536  884736     4988151      NaN      NaN
3             143       2           1      NaN      NaN
4             652      20           1      NaN      NaN
AnnData object with n_obs × n_vars = 884736 × 4536
    obs: 'barcode'
    var: 'gene_id'
                                               barcode
AAACATCG_AAACATCG_AAACGATA  AAACATCG_AAACAT

In [6]:

print(barcodes.head())  # Vérifie les premiers barcodes (cellules)
print(features.head())  # Vérifie les premiers gènes

0    AAACATCG_AAACATCG_AAACGATA
1    AACAACCA_AAACATCG_AAACGATA
2    AACCGAGA_AAACATCG_AAACGATA
3    AACGCTTA_AAACATCG_AAACGATA
4    AACGTGAT_AAACATCG_AAACGATA
Name: 0, dtype: object
           0          1                2
0  BSU_00010  BSU_00010  Gene Expression
1  BSU_00020  BSU_00020  Gene Expression
2  BSU_00030  BSU_00030  Gene Expression
3  BSU_00040  BSU_00040  Gene Expression
4  BSU_00050  BSU_00050  Gene Expression


In [14]:
import os
#print(os.getcwd())  # Affiche le répertoire de travail actuel
os.makedirs("7_Article/results/bacteria_growth", exist_ok=True)  # Crée le dossier s'il n'existe pas
adata.write("7_Article/results/bacteria_growth/data.h5ad")  # Sauvegarde dans le dossier "results"
print(os.path.exists("7_Article/results/bacteria_growth/data.h5ad"))  # True si le fichier est bien enregistré

True


In [16]:
# Charger les données
adata = sc.read("7_Article/results/bacteria_growth/data.h5ad")
# Afficher les dimensions
pp.pprint(adata)  # Affiche les dimensions de la matrice d'expression

# Afficher les 16 premières cellules et les 10 premiers gènes
pp.pprint(adata[:16, :10].to_df())


AnnData object with n_obs × n_vars = 884736 × 4536
    obs: 'barcode'
    var: 'gene_id'
                            BSU_00010  BSU_00020  BSU_00030  BSU_00040  \
AAACATCG_AAACATCG_AAACGATA        0.0        0.0        0.0        0.0   
AACAACCA_AAACATCG_AAACGATA        0.0        0.0        0.0        0.0   
AACCGAGA_AAACATCG_AAACGATA        0.0        0.0        0.0        0.0   
AACGCTTA_AAACATCG_AAACGATA        0.0        0.0        0.0        0.0   
AACGTGAT_AAACATCG_AAACGATA        0.0        0.0        0.0        0.0   
AACTCACC_AAACATCG_AAACGATA        0.0        0.0        0.0        0.0   
AAGACGGA_AAACATCG_AAACGATA        0.0        0.0        0.0        0.0   
AAGAGATC_AAACATCG_AAACGATA        0.0        0.0        0.0        0.0   
AAGGACAC_AAACATCG_AAACGATA        0.0        0.0        0.0        0.0   
AAGGTACA_AAACATCG_AAACGATA        0.0        0.0        0.0        0.0   
AATCCGTC_AAACATCG_AAACGATA        0.0        0.0        0.0        0.0   
AATGTTGC_AAACATCG_AAACG